In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comspscholar-dataset/Brain Dead CompScholar Dataset.csv


In [10]:
data=pd.read_csv('/kaggle/input/comspscholar-dataset/Brain Dead CompScholar Dataset.csv')
data.head()

,Paper Id,Paper Title,Key Words,Abstract,Conclusion,Document,Paper Type,Summary,Topic,OCR,labels
0,1,Multi-document Summarization via Deep Learning...,"Multi-document summarization (MDS), Deep learn...",Multi-document summarization (MDS) is an effec...,"In this article, we have presented the first c...",Multi-document Summarization via Deep Learning...,Text summarization,This article presents a systematic overview of...,Natural Language Processing,"lla i aye RR | poe [Sena Sena | Sena, —+ ar ea...",Deep Learning and Machine Learning
1,2,NLP based Machine Learning Approaches for Text...,"Text summarization, Abstractive and extractive...","Due to the plethora of data available today, t...",We have seen that due to abundant availability...,NLP based Machine Learning Approaches for Text...,Natural Language Processing,The article discusses the importance of text s...,Natural Language Processing,@STOM © Word Vector Embedding kenearest neighb...,Deep Learning and Machine Learning
2,3,Abstractive text summarization using LSTM-CNN ...,Text mining . Abstractive text summarization ....,"Abstractive Text Summarization (ATS), which i...","In this paper, we develop a novel LSTM-CNN bas...",Abstractive text summarization using LSTM-CNN ...,Text summarization,The article presents a new framework for abstr...,Natural Language Processing,encoder decoderWord Merpholosical Coreterence ...,Deep Learning and Machine Learning
3,4,DEXPERTS: Decoding-Time Controlled Text Genera...,"Natural language generation, Controlled text g...",Despite recent advances in natural language\ng...,"We present DEXPERTS, a method for controlled\n...",DEXPERTS: Decoding-Time Controlled Text Genera...,Text generation,The paper proposes a method called DEXPERTS fo...,Natural Language Processing,reatva star on negative proms oe TT os ee oe S...,Deep Learning and Machine Learning
4,5,A Survey of Knowledge-enhanced Text Generation,"text-to-text generation, natural language proc...",The goal of text-to-text generation is to make...,"In this survey, we present a comprehensive rev...",A Survey of Knowledge-enhanced Text Generation...,Text generation,The paper discusses the challenges in text-to-...,Natural Language Processing,(ira => Generation model => foam] | Generation...,Deep Learning and Machine Learning


In [ ]:
summary = data.loc[0, "Summary"]
abstract = data.loc[0, "Abstract"]

# Printing the values
print("Summary:", summary)
print("Abstract:", abstract)

In [ ]:
import nltk

# Ensure nltk's tokenizer is available
nltk.download('punkt')
from nltk.tokenize import word_tokenize

tokens = word_tokenize(data.loc[266, "Summary"])

# Counting the number of tokens
token_count = len(tokens)

print("Number of tokens in Summary:", token_count)

In [ ]:
row=data.loc[0]
text = " ".join(row[["Paper Title", "Abstract", "Key Words", "Document", "Conclusion", "Paper Type"]].astype(str))
tokens=word_tokenize(text)
token_count = len(tokens)

print("Number of tokens in Summary:", token_count)

In [ ]:
maxi=0
pos=0
for index, row in data.iterrows():
    # text = " ".join(row[["Paper Title", "Abstract", "Key Words", "Document", "Conclusion", "Paper Type"]].astype(str))
    text=row["Summary"]
    tokens=word_tokenize(text)
    token_count = len(tokens)
    # if(token_count>maxi):
    #     maxi=token_count
    #     pos=index
    maxi+=token_count
print(maxi/371)

In [46]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [47]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d246b3cd6452dbe1ec3262ba26665042f0e149eba28a903041b01d404cea5967
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [8]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
torch.cuda.empty_cache()

In [23]:
import torch
from tqdm import tqdm

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches that we can process simultaneously.
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device="cuda"):
    """
    Compute the evaluation metric for the dataset.
    """
    # Preparing input and target texts
    dataset_inputs = dataset["Text"].fillna(" ").tolist()
    dataset_targets = dataset["Summary"].fillna(" ").tolist()  # Ensure summaries are valid strings
    
    # Splitting into batches
    input_batches = list(generate_batch_sized_chunks(dataset_inputs, batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset_targets, batch_size))

    # Processing each batch
    for input_batch, target_batch in tqdm(zip(input_batches, target_batches), 
                                          total=len(input_batches), desc="Processing Batches"):

        # Tokenizing the input text
        inputs = tokenizer(input_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        # Generating summaries
        with torch.no_grad():  # Disable gradient computation for inference
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8, num_beams=8, max_length=128)

        # Decoding the summaries
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                             for s in summaries]

        # Updating the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute and return the final scores
    score = metric.compute()
    return score


In [9]:
from sklearn.model_selection import train_test_split
import datasets

cols = ["Paper Title", "Key Words", "Abstract", "Conclusion", "Document", "Paper Type", "Topic"]

# Ensure missing values are replaced with empty strings before concatenation
data["Text"] = data[cols].fillna(" ").agg(" ".join, axis=1)
data=data[["Text", "Summary"]]
# Splitting data into train (70%), validation (15%), and test (15%)
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = datasets.Dataset.from_pandas(train_data)
val_dataset = datasets.Dataset.from_pandas(val_data)
test_dataset = datasets.Dataset.from_pandas(test_data)

In [10]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Text'], max_length=1024, truncation=True)

# Tokenizing the Target Summaries:
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['Summary'], max_length = 256, truncation = True )

# Returning the Tokenized Features:
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

# Apply the function to datasets
train_dataset = train_dataset.map(convert_examples_to_features, batched=True)
val_dataset = val_dataset.map(convert_examples_to_features, batched=True)
test_dataset = test_dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/259 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [11]:
columns = ["input_ids", "labels", "attention_mask"]
train_dataset.set_format(type="torch", columns=columns)
val_dataset.set_format(type="torch", columns=columns)
test_dataset.set_format(type="torch", columns=columns)

In [12]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments( output_dir="pegasus-samsum",
                                 num_train_epochs=10,
                                 warmup_steps=500,
                                 per_device_train_batch_size=1,
                                 per_gpu_eval_batch_size=1,
                                 weight_decay=0.01,
                                 logging_steps=10, 
                                 push_to_hub=True,
                                 evaluation_strategy='steps', 
                                 eval_steps=500,
                                 save_steps=1e6,
                                 gradient_accumulation_steps=16,
                                 report_to="none")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
len(tokenizer)

96103

In [27]:
trainer = Trainer(model=model_pegasus, 
                 args=training_args,
                 tokenizer=tokenizer,
                 data_collator=seq2seq_data_collator,
                 train_dataset=train_dataset,
                 eval_dataset=val_dataset)

<ipython-input-27-8ca8c94c3db3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus,


In [28]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=160, training_loss=1.7023686051368714, metrics={'train_runtime': 957.9728, 'train_samples_per_second': 2.704, 'train_steps_per_second': 0.167, 'total_flos': 6320455092731904.0, 'train_loss': 1.7023686051368714, 'epoch': 9.432432432432432})

In [29]:
#model with fine-tuning
metric = evaluate.load("rouge")
score = calculate_metric_on_test_ds(test_data, metric, trainer.model, tokenizer, batch_size=4, device=device)
print(score)

Processing Batches: 100%|██████████| 14/14 [01:08<00:00,  4.90s/it]


{'rouge1': 0.4926840042646487, 'rouge2': 0.2624993226301624, 'rougeL': 0.35761643289405654, 'rougeLsum': 0.3576101533213595}


In [25]:
#model without fine tuning
metric = evaluate.load("rouge")
score = calculate_metric_on_test_ds(test_data, metric, model_pegasus, tokenizer, batch_size=4, device=device)
print(score)

Processing Batches: 100%|██████████| 14/14 [00:53<00:00,  3.80s/it]


{'rouge1': 0.34926607845636115, 'rouge2': 0.17728131279827714, 'rougeL': 0.2512846517424161, 'rougeLsum': 0.2505077608020769}


In [30]:
!zip -r /kaggle/working/output.zip /kaggle/working/


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/pegasus-samsum/ (stored 0%)
  adding: kaggle/working/pegasus-samsum/checkpoint-160/ (stored 0%)
  adding: kaggle/working/pegasus-samsum/checkpoint-160/optimizer.pt^C



zip error: Interrupted (aborting)


In [ ]:
from IPython.display import FileLink
FileLink('/kaggle/working/output.zip')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
from sklearn.model_selection import train_test_split
import datasets

data=pd.read_csv('/kaggle/input/comspscholar-dataset/Brain Dead CompScholar Dataset.csv')
cols = ["Paper Title", "Key Words", "Abstract", "Conclusion", "Document", "Paper Type", "Topic"]

# Ensure missing values are replaced with empty strings before concatenation
data["Text"] = data[cols].fillna(" ").agg(" ".join, axis=1)
data=data[["Text", "Summary"]]
# Splitting data into train (70%), validation (15%), and test (15%)
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = datasets.Dataset.from_pandas(train_data)
val_dataset = datasets.Dataset.from_pandas(val_data)
test_dataset = datasets.Dataset.from_pandas(test_data)

In [19]:
data.head()

,Text,Summary
0,Multi-document Summarization via Deep Learning...,This article presents a systematic overview of...
1,NLP based Machine Learning Approaches for Text...,The article discusses the importance of text s...
2,Abstractive text summarization using LSTM-CNN ...,The article presents a new framework for abstr...
3,DEXPERTS: Decoding-Time Controlled Text Genera...,The paper proposes a method called DEXPERTS fo...
4,A Survey of Knowledge-enhanced Text Generation...,The paper discusses the challenges in text-to-...


In [11]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers triton
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

import os
os.environ["WANDB_DISABLED"] = "true"

In [13]:
%%capture
!pip install xformers

In [73]:
# For dataset
import pandas as pd
import json
import os
import ast
import re
import numpy as np
from datasets import Dataset
import matplotlib.pyplot as plt

# For LLM
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)
from trl import SFTTrainer, setup_chat_format, SFTConfig

import torch

set_seed(42)

In [74]:
from unsloth import FastLanguageModel

In [75]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model_path="/kaggle/input/fine-tuned-llama-model"
model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2",
    model_name="unsloth/Meta-Llama-3.1-8B",
    # model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [76]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [77]:
from datasets import Dataset
prompt="""Below is an instruction that describes a task, Paired with an input
### Instruction:
Analyze the research article content and give me a summary from the research article. The summary should be as precise as possible and must include all the important points. Donot repeat information. Make the summary as crisp as possible

### Input:
{}
### Response: 
{}
"""

EOS_TOKEN=tokenizer.eos_token
def formatting_prompts_func(row):
    inputs =row["Text"]
    outputs = row["Summary"]
    texts = []
    
    for input, output in zip(inputs, outputs):
        text = prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
        
    return { "texts" : texts, }

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/259 [00:00<?, ? examples/s]

In [78]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "texts",
    max_seq_length = 1024,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        max_steps = 35,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["texts"] (num_proc=2):   0%|          | 0/259 [00:00<?, ? examples/s]

In [79]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla P100-PCIE-16GB. Max memory = 15.888 GB.
12.521 GB of memory reserved.


In [80]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 259 | Num Epochs = 2 | Total steps = 35
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.410200
2,1.439800
3,1.636100
4,1.586500
5,1.388900
6,1.437700
7,1.255400
8,1.103200
9,1.319700
10,1.464800


In [81]:
trainer.model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [31]:
text_str = test_data.loc[[0]]['Text'].iloc[0]
text_str

'Multi-document Summarization via Deep Learning\nTechniques: A Survey Multi-document summarization (MDS), Deep learning models, Objective functions, Taxonomy, Evaluation metrics, Future directions, Information aggregation Multi-document summarization (MDS) is an effective tool for information aggregation that generates an in\x02formative and concise summary from a cluster of topic-related documents. Our survey, the first of its kind,\nsystematically overviews the recent deep-learning-based MDS models. We propose a novel taxonomy to sum\x02marize the design strategies of neural networks and conduct a comprehensive summary of the state of the\nart. We highlight the differences between various objective functions that are rarely discussed in the existing\nliterature. Finally, we propose several future directions pertaining to this new and exciting field. In this article, we have presented the first comprehensive review of the most notable works to date\non deep-learning-based multi-docume

In [35]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        text_str,
        ""
    )
], return_tensors="pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens = 128)
generated_text = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)

# Extract the response by looking for the "### Response:" part
response_start = generated_text.find("### Response:") + len("### Response:")
response = generated_text[response_start:].strip()
response

<|begin_of_text|>Below is an instruction that describes a task, Paired with an input
### Instruction:
Analyze the research article content and give me a summary from the research article. The summary should be as precise as possible and must include all the important points. Donot repeat information. Make the summary as crisp as possible

### Input:
Multi-document Summarization via Deep Learning
Techniques: A Survey Multi-document summarization (MDS), Deep learning models, Objective functions, Taxonomy, Evaluation metrics, Future directions, Information aggregation Multi-document summarization (MDS) is an effective tool for information aggregation that generates an informative and concise summary from a cluster of topic-related documents. Our survey, the first of its kind,
systematically overviews the recent deep-learning-based MDS models. We propose a novel taxonomy to summarize the design strategies of neural networks and conduct a comprehensive summary of the state of the
art. We 

''

In [82]:
from transformers import TextStreamer

def get_response_from_model(row, model, tokenizer, prompt_template):
    # Prepare the input using the row's 'Text'
    prompt = prompt_template.format(
        row.iloc[0]['Text'],  # Properly extract text from the DataFrame
        ""
    )

    # Tokenize and prepare inputs for the model
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # # Set up the TextStreamer to process the model's output
    # text_streamer = TextStreamer(tokenizer)

    # Generate the response (this waits for model completion)
    output_ids = model.generate(**inputs, max_new_tokens=128)

    # Decode the generated output
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(generated_text)
    # Extract the response after "### Response:"
    response_start = generated_text.find("### Response:") + len("### Response:")
    response = generated_text[response_start:].strip() if response_start != -1 else generated_text

    return response

# Call the function
response = get_response_from_model(test_data.loc[[0]], trainer.model, tokenizer, prompt)
print("\nFinal Response:\n", response)

Below is an instruction that describes a task, Paired with an input
### Instruction:
Analyze the research article content and give me a summary from the research article. The summary should be as precise as possible and must include all the important points. Donot repeat information. Make the summary as crisp as possible

### Input:
Multi-document Summarization via Deep Learning
Techniques: A Survey Multi-document summarization (MDS), Deep learning models, Objective functions, Taxonomy, Evaluation metrics, Future directions, Information aggregation Multi-document summarization (MDS) is an effective tool for information aggregation that generates an informative and concise summary from a cluster of topic-related documents. Our survey, the first of its kind,
systematically overviews the recent deep-learning-based MDS models. We propose a novel taxonomy to summarize the design strategies of neural networks and conduct a comprehensive summary of the state of the
art. We highlight the dif

In [83]:
import evaluate

# Load the ROUGE metric
metric = evaluate.load("rouge")

# Ensure predictions and references are lists
predictions = [response]  # Wrap response in a list
references = [test_data.loc[[0]]['Summary'].values[0]]  # Wrap reference in a list

# Add batch for evaluation
metric.add_batch(predictions=predictions, references=references)

# Compute the score
score = metric.compute()

# Return or print the score
print(score)

{'rouge1': 0.5340314136125655, 'rouge2': 0.2857142857142857, 'rougeL': 0.418848167539267, 'rougeLsum': 0.418848167539267}


In [86]:
from transformers import TextStreamer
import evaluate

test_data=temp_data

def get_response_from_model(row, model, tokenizer, prompt_template):
    # Prepare the input using the row's 'Text'
    prompt = prompt_template.format(
        row['Text'],  # Extract text
        ""
    )

    # Tokenize and prepare inputs for the model
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the response
    output_ids = model.generate(**inputs, max_new_tokens=128)

    # Decode the generated output
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract response after "### Response:"
    response_start = generated_text.find("### Response:") + len("### Response:")
    response = generated_text[response_start:].strip() if response_start != -1 else generated_text

    return response

# Load ROUGE metric
metric = evaluate.load("rouge")
print("Length of test data",len(test_data))
# List to store responses
responses = []
cnt=0;
# Iterate over all rows in test_data
for _, row in test_data.iterrows():
    print(cnt);
    cnt+=1;
    response = get_response_from_model(row, trainer.model, tokenizer, prompt)
    responses.append(response)

# Ensure predictions and references are lists
predictions = responses
references = test_data["Summary"].tolist()  # Convert 'Summary' column to list

# Add batch for evaluation
metric.add_batch(predictions=predictions, references=references)

# Compute the ROUGE score
score = metric.compute()

# Print results
print("\nFinal ROUGE Score:\n", score)

Length of test data 112
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111

Final ROUGE Score:
 {'rouge1': 0.5720591675557505, 'rouge2': 0.3409949041021048, 'rougeL': 0.42209499308107934, 'rougeLsum': 0.4218762224603859}
